In [2]:
import sys, os
DMOL_DIR = "/Users/Henryye/research/shaf/DMOL"
sys.path.append(os.path.join(DMOL_DIR, "utils"))

#from ml_utlis import *
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '/Users/CalebPitts/Documents/Files/School/College/19-20-Year/Research/DMOL/utils')
#from ml_utils import *

import numpy as np
import pandas as pd
pd.options.display.max_columns = 999

import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.decomposition import PCA

import warnings
warnings.filterwarnings('ignore')

In [3]:
df_proc = pd.read_csv("/Users/CalebPitts/Documents/Files/School/College/19-20-Year/Research/data/proc_df.csv")
var_map = load_object("/Users/CalebPitts/Documents/Files/School/College/19-20-Year/Research/data/var_map.pkl")

NameError: name 'load_object' is not defined

In [4]:
def train_model(df, week_num, n_comp = None, use_fr = False, plot = False, verbose = False):
    
    def construct_X(df, var_map_curr, n_comp = None, verbose = False):
        X = pd.DataFrame()
        pca = PCA(n_components=n_comp) if n_comp is not None else None
        print(pca.components_)
        for construct in var_map_curr["Quant"]:
            temp = df[list(var_map_curr["Quant"][construct])].copy()
            temp = temp.fillna(temp.mode().iloc[0])
            if n_comp is None:
                temp_mean = pd.DataFrame(temp.mean(axis=1))
                temp_mean.columns = ["{}_mean".format(construct)]
                X = pd.concat([X, temp_mean], axis = 1)
            else:
                if len(var_map_curr["Quant"][construct]) < n_comp:
                    X = pd.concat([X, temp.copy()], axis = 1)
                else:
                    temp_pca = pd.DataFrame(pca.fit_transform(temp))
                    temp_pca.columns = ["{}_PC{}".format(construct, i) for i in range(1, n_comp+1)]
                    X = pd.concat([X, temp_pca], axis = 1)
                    if verbose:
                        print("{} variance explained:\n{}\n".format(construct, pca.explained_variance_ratio_))
        return X
    
    var_map_curr = NEW_VAR_MAP["week_{}".format(week_num)]
    X, y = construct_X(df, var_map_curr, n_comp, verbose), df["gr_revq{}_quantile".format(week_num)]
    for gr in ["gr_revq{}".format(i) for i in range(max(1, week_num-1), week_num)]:
        X[gr] = df[gr]
    
    ### Classification Modeling ###
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    
    for n in range(5, 9):
        print("\nN Estimators:", n)
        mod = RandomForestClassifier(n_estimators=n, max_depth=None).fit(X_train, y_train)
    
        y_train_pred = mod.predict(X_train)
        print("Training Accuracy: {}".format(accuracy_score(y_train, y_train_pred)))

        y_test_pred = mod.predict(X_test)
        print("Testing  Accuracy: {}".format(accuracy_score(y_test, y_test_pred)))

In [ ]:
for week_num in range(1, 6):
    print("Week {}".format(week_num))
    train_model(df_proc, week_num, use_fr = False, plot = False)
    print('\n\n')